In [3]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.0.0
Eager mode:  True
Hub version:  0.11.0
GPU is NOT AVAILABLE


In [4]:
path='sentiment labelled sentences\\sentiment labelled sentences'
datafiles=[os.path.join(path,file) for file in os.listdir(path) if 'labelled' in file ]
import pandas as pd
dataframe=[]
for file in datafiles:
    data=pd.read_csv(file,sep='\t',header=None)
    data.columns=['Phrase','Sentiment']
    dataframe.append(data)
data=pd.concat(dataframe).reset_index(drop=True)
data['Sentiment']=data['Sentiment'].apply(lambda x: 'Negative' if x==0 else 'Positive')
data

,Phrase,Sentiment
0,So there is no way for me to plug it in here i...,Negative
1,"Good case, Excellent value.",Positive
2,Great for the jawbone.,Positive
3,Tied to charger for conversations lasting more...,Negative
4,The mic is great.,Positive
...,...,...
2743,I think food should have flavor and texture an...,Negative
2744,Appetite instantly gone.,Negative
2745,Overall I was not impressed and would not go b...,Negative
2746,"The whole experience was underwhelming, and I ...",Negative


In [16]:
from sklearn.preprocessing import OneHotEncoder
type_one_hot = OneHotEncoder(sparse=False).fit_transform(
  data.Sentiment.to_numpy().reshape(-1, 1)
)
type_one_hot.shape

## Or

Y=data.Sentiment

from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
type_one_hot=lb.fit_transform(Y)
print (type_one_hot.shape,lb.inverse_transform(type_one_hot))
len(lb.classes_),lb.classes_,type_one_hot.shape

(2748, 1) ['Negative' 'Positive' 'Positive' ... 'Negative' 'Negative' 'Negative']


(2, array(['Negative', 'Positive'], dtype='<U8'), (2748, 1))

In [6]:
# features=np.array(data[['Activity name']]).reshape(-1,)

In [17]:
from sklearn.model_selection import train_test_split
features=np.array(data[['Phrase']]).reshape(-1,)
import numpy as np
train_reviews, test_reviews, y_train, y_test =\
  train_test_split(
    features,
    type_one_hot,
    test_size=.1,stratify=type_one_hot,
    random_state=7
  )
train_reviews.shape
#Test Stratified
print (pd.DataFrame(lb.inverse_transform(y_test),columns=['Class']).groupby('Class').size())
#Data Stratified
pd.DataFrame(lb.inverse_transform(type_one_hot),columns=['Class']).groupby('Class').size()

Class
Negative    136
Positive    139
dtype: int64


Class
Negative    1362
Positive    1386
dtype: int64

In [28]:
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=False)

In [29]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1,activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 128)               124642688 
_________________________________________________________________
dense_8 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 17        
Total params: 124,644,769
Trainable params: 2,081
Non-trainable params: 124,642,688
_________________________________________________________________


In [30]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy','Precision','Recall'])

In [31]:
#train_reviews, test_reviews, y_train, y_test
history = model.fit(x=train_reviews,y=y_train,validation_data=(test_reviews,y_test),
                    epochs=2,
                    verbose=1)

Train on 2473 samples, validate on 275 samples
Epoch 1/2
2473/2473 [==============================] - 1s 423us/sample - loss: 7.6015 - accuracy: 0.5042 - Precision: 0.5042 - Recall: 1.0000 - val_loss: 7.5830 - val_accuracy: 0.5055 - val_Precision: 0.5055 - val_Recall: 1.0000
Epoch 2/2
2473/2473 [==============================] - 0s 68us/sample - loss: 7.6015 - accuracy: 0.5042 - Precision: 0.5042 - Recall: 1.0000 - val_loss: 7.5830 - val_accuracy: 0.5055 - val_Precision: 0.5055 - val_Recall: 1.0000


In [ ]:
# test_reviews[0:10],y_test[0:10]

In [ ]:
results = model.evaluate(test_reviews[0:1000],y_test[0:1000],verbose=0)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

In [ ]:
Halt..

In [ ]:
results=np.argmax(model.predict(test_reviews),1)+1
lb.inverse_transform(model.predict(test_reviews)),lb.inverse_transform(y_test),lb.classes_,results

In [ ]:
#All data fit.
#features=np.array(data[['Activity name']]).reshape(-1,)
history = model.fit(x=features,y=type_one_hot,
                    epochs=30,
                    verbose=1)

In [ ]:
Halt..

In [ ]:
# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5',custom_objects={'KerasLayer':hub.KerasLayer})

# Show the model architecture
new_model.summary()

In [ ]:
lb.inverse_transform(new_model.predict(test_data))

In [ ]:
def custom_predict(pandas_single_textcolumn_as_data_frame_input,sk_learn_label_binarizer_object,model_h5file):
    # Recreate the exact same model, including its weights and the optimizer
    new_model = tf.keras.models.load_model(model_h5file,custom_objects={'KerasLayer':hub.KerasLayer})
    testdata=np.array(pandas_single_textcolumn_as_data_frame_input).reshape(-1,)
    predictions=sk_learn_label_binarizer_object.inverse_transform(new_model.predict(testdata))
    return predictions
custom_predict(test[["Phrase"]],lb,model_h5file='my_model.h5')